# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [1]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("Alpha.ipynb");

In [2]:
function read_training_alpha(alpha, split)
    uid = parse(Int, get_recommendee_username()) + 1
    df = read_alpha(alpha, split, true)
    mask = df.user .== uid
    df = RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    df.user .= 1
    df
end;

In [3]:
function average_errors(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    abs.(training.rating .- serving.rating[training.item])
end

function relative_errors(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    ϵ = eps(Float64)
    abs.(abs.(training.rating) ./ (abs.(serving.rating[training.item]) .+ ϵ) .- 1)
end;

In [4]:
function maximum_error(alpha, split)
    maximum(vcat(average_errors(alpha, split), relative_errors(alpha, split)))
end

function average_abs_error(alpha, split)
    mean(average_errors(alpha, split))
end

function average_rel_error(alpha, split)
    mean(relative_errors(alpha, split))
end;

In [5]:
function display_errors(alphas, split)
    alpha_padding = maximum(length.(alphas))
    number_padding = 16

    header_1 = rpad("Alpha", alpha_padding)
    header_2 = rpad("Avg Abs Error", number_padding)
    header_3 = rpad("Avg Rel Error", number_padding)
    header_4 = rpad("Max Error", number_padding)
    @info "$header_1 $header_2 $header_3 $header_4"

    for alpha in alphas
        avg_abs_error = rpad(average_abs_error(alpha, split), number_padding)
        avg_rel_error = rpad(average_rel_error(alpha, split), number_padding)
        max_error = rpad(maximum_error(alpha, split), number_padding)
        alpha = rpad(alpha, alpha_padding)
        @info "$alpha $avg_abs_error $avg_rel_error $max_error"
    end
end;

In [6]:
potential_alphas = readdir(recommendee_alpha_basepath())
alphas = sort([
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
])
ensemble_alphas = sort(["ErrorExplicit", "ErrorImplicit", "Explicit", "NonlinearExplicit"])
inference_only_alphas = sort(["ItemCFRelated", "BPR"])
standard_alphas = [x for x in alphas if x ∉ ensemble_alphas && x ∉ inference_only_alphas];

In [7]:
display_errors(standard_alphas, "training")
# ensemble alphas do not save training splits

[ Info: 20220704 16:58:15 Alpha                             Avg Abs Error    Avg Rel Error    Max Error       
[ Info: 20220704 16:58:23 ExplicitItemCF                    9.706281e-7      0.05265420916290078 1.0             
[ Info: 20220704 16:58:27 ExplicitUserItemBiases            1.9073486e-6     2.3695841768580756e-7 1.9073486328125e-6
[ Info: 20220704 16:58:32 LinearExplicit                    0.0008248028     9.900572676129147e-5 0.0020475387573242188
[ Info: 20220704 16:58:37 LinearImplicit                    5.483985e-6      0.0015382511445828945 0.009135707927428571
[ Info: 20220704 16:58:42 NeuralExplicitAutoencoderUntuned  0.00057415804    0.0034894848548399113 0.04958087969790004
[ Info: 20220704 16:58:47 NeuralExplicitItemCFUntuned       0.00029703407    0.0024231337060731865 0.017875267752952784
[ Info: 20220704 16:58:51 NeuralExplicitMatrixFactorization 0.024161706      0.16312787731205505 3.238270371979441
[ Info: 20220704 16:58:56 NeuralImplicitAutoencoderUntuned  5.2

In [8]:
display_errors(standard_alphas, "validation")
println()
display_errors(ensemble_alphas, "validation")

[ Info: 20220704 16:59:15 Alpha                             Avg Abs Error    Avg Rel Error    Max Error       
[ Info: 20220704 16:59:16 ExplicitItemCF                    1.0281801e-6     1.3570750168057089e-6 1.7881393432617188e-6
[ Info: 20220704 16:59:17 ExplicitUserItemBiases            1.9073486e-6     2.456523182647885e-7 1.9073486328125e-6
[ Info: 20220704 16:59:17 LinearExplicit                    0.0014333725     0.00017198708015003783 0.0015726089477539062
[ Info: 20220704 16:59:18 LinearImplicit                    1.5850528e-5     0.0016329078093766936 0.0031676523257262446
[ Info: 20220704 16:59:19 NeuralExplicitAutoencoderUntuned  0.0004582815     0.07252747807446225 0.14472584085701246
[ Info: 20220704 16:59:19 NeuralExplicitItemCFUntuned       0.00045789778    0.0012813613982458616 0.0019024471491065897
[ Info: 20220704 16:59:20 NeuralExplicitMatrixFactorization 0.0332275        0.22907921850031332 0.40753384317851515
[ Info: 20220704 16:59:20 NeuralImplicitAutoencoderUn

[ Info: 20220704 16:59:23 ErrorExplicit     0.052846134      0.08406199477224358 0.16800208237958947
[ Info: 20220704 16:59:24 ErrorImplicit     0.0070433766     0.022194990831971573 0.022604964932871896
[ Info: 20220704 16:59:25 Explicit          0.023903847      0.002768496656071595 0.040261268615722656
[ Info: 20220704 16:59:25 NonlinearExplicit 0.025337307      0.3317311669483993 0.5311754038744756


In [9]:
display_errors(standard_alphas, "test")
println()
display_errors(ensemble_alphas, "test")

[ Info: 20220704 16:59:25 Alpha                             Avg Abs Error    Avg Rel Error    Max Error       
[ Info: 20220704 16:59:26 ExplicitItemCF                    6.748097e-7      2.185886216619716e-6 5.327008317124182e-6
[ Info: 20220704 16:59:27 ExplicitUserItemBiases            1.9073486e-6     2.6323616132893436e-7 1.9073486328125e-6
[ Info: 20220704 16:59:28 LinearExplicit                    0.0005258833     7.119812936281658e-5 0.0010466575622558594
[ Info: 20220704 16:59:28 LinearImplicit                    4.4420344e-7     0.00022590453380964628 0.0008153138975939278
[ Info: 20220704 16:59:29 NeuralExplicitAutoencoderUntuned  0.00089455955    0.005554864799354821 0.015459151297960494
[ Info: 20220704 16:59:30 NeuralExplicitItemCFUntuned       0.00016586749    0.0019774518640843152 0.007118796318736242
[ Info: 20220704 16:59:30 NeuralExplicitMatrixFactorization 0.022871813      0.21341033276212573 0.5903258653159711
[ Info: 20220704 16:59:31 NeuralImplicitAutoencoderUntu

[ Info: 20220704 16:59:35 ErrorExplicit     0.0020524007     0.0021366202920438987 0.010204664458911772
[ Info: 20220704 16:59:35 ErrorImplicit     0.00030841146    0.0010182613266569757 0.006597458066771811
[ Info: 20220704 16:59:36 Explicit          0.0015469278     0.00021132068925541362 0.004282474517822266
[ Info: 20220704 16:59:36 NonlinearExplicit 0.0012913415     0.049060484164935546 0.1493740640319665
